# AES_Adv_1

In SCA101, we focused only on a single attack location, the SBox, without much explanation or experimentation. In this lab, we'll be trying some other attack locations to better understand why the SBox was used exclusively in SCA101.

## Capture

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
N = 200  # Number of traces
project = cw.create_project("AES_Adv_1.cwp")

for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

#Convert traces to numpy arrays

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

In [ ]:
print(results.key_guess())

In [ ]:
leak_model.key_schedule_rounds(results.key_guess(), 0, 10)

In [ ]:
plot_data = cwa.analyzer_plots(results)
def byte_to_color(idx):
    return hv.Palette.colormaps['Category20'](idx/16.0)

import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
import pandas as pd, numpy as np

a = []
b = []
hv.extension('bokeh')
for i in range(0, 16):
    data = plot_data.output_vs_time(i)
    a.append(np.array(data[1]))
    b.append(np.array(data[2]))
    b.append(np.array(data[3]))
    
pda = pd.DataFrame(a).transpose().rename(str, axis='columns')
pdb = pd.DataFrame(b).transpose().rename(str, axis='columns')
curve = hv.Curve(pdb['0'], "Sample").options(color='black')
for i in range(1, 16):
    curve *= hv.Curve(pdb[str(i)]).options(color='black')
    
for i in range(0, 16):
    curve *= hv.Curve(pda[str(i)]).options(color=byte_to_color(i))
decimate(curve.opts(width=900, height=600))

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

In [ ]:
leak_model = cwa.leakage_models.plaintext_key_xor

In [ ]:
print(cwa.leakage_models)

In [ ]:
attack = cwa.cpa(project, leak_model)
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

At first glance, our attack appears to have partially failed, recovering roughly half the key bytes. This isn't quite true: we've got two equal key guesses for each byte. Remember back to the "recovering data from a single bit" slides: the correct key and its inverse will produce inverted results. This means that they're both have the same correlation with the power trace, just that one is negative and one is positive.

So we have two possible guesses for each key. Is this enough to stop us from recovering the key? In this case, it's pretty easy to brute force the correct key. In an actual attack, we might not know the ciphertext, making it much harder to bruteforce the key.

The XOR operation is also linear, meaning incorrect key guesses have higher correlations than they would with a non-linear operation like the SBox. If we didn't have as nice of a measurement environment, this might make the difference between being able to tell we've got the key and not.

In [ ]:
plot_data = cwa.analyzer_plots(results)
def byte_to_color(idx):
    return hv.Palette.colormaps['Category20'](idx/16.0)

import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
import pandas as pd, numpy as np

a = []
b = []
hv.extension('bokeh')
for i in range(0, 16):
    data = plot_data.output_vs_time(i)
    a.append(np.array(data[1]))
    b.append(np.array(data[2]))
    b.append(np.array(data[3]))
    
pda = pd.DataFrame(a).transpose().rename(str, axis='columns')
pdb = pd.DataFrame(b).transpose().rename(str, axis='columns')
curve = hv.Curve(pdb['0'], "Sample").options(color='black')
for i in range(1, 16):
    curve *= hv.Curve(pdb[str(i)]).options(color='black')
    
for i in range(0, 16):
    curve *= hv.Curve(pda[str(i)]).options(color=byte_to_color(i))
decimate(curve.opts(width=900, height=600))

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.new_model(ShiftRowsOutput)
attack = cwa.cpa(project, leak_model)
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

For the shift rows output, we can see that the attack has again failed. This time, it's much worse: we've recovered only 4 bytes. All the rest have completely failed! You might've noticed something odd about the bytes that failed: every key guess has exactly the same correlation! To understand this, we'll need to look at what the shift rows instruction actuall does. As the name suggests, it shifts the rows of the AES state around, more specifically, it turns the following matrix:

$$
state = \left( \begin{array}
& S0 & S4 & S8 & S12 \\
S1 & S5 & S9 & S13 \\
S2 & S6 & S10 & S14 \\
S3 & S7 & S11 & S15
\end{array} \right)
$$

into this one:

$$
state = \left( \begin{array}
& S0 & S4 & S8 & S12 \\
S5 & S9 & S13 & S1 \\
S10 & S14 & S2 & S6 \\
S15 & S3 & S7 & S11
\end{array} \right)
$$

Except for the top row, for which our guess succeeded, the bytes don't align anymore. This isn't technically a difference that should affect anything in our attack. Analyzer, however, only considers one byte key guess at a time and assumes that all the others are zero. This means the leakage was always calculated for a key byte of 0. We could fix this in the leakage model, but really there's no reason to: shift rows is just moving values around, it's not changing anything. This means we don't really learn anything new as compared to the SBox output, at least for a software AES attack. This also means that our regular leakage model will still pick up the SBox operation - the keys will just be shifted from what we would expect.

A similar problem affects the next operation in AES, mix columns, with a similarly simple fix. This is covered in our article on extending AES128 attacks to AES256: https://wiki.newae.com/Extending_AES-128_Attacks_to_AES-256. Basically, since Mix Columns is a linear operation, we can continue to use our SBox output. The key we recover, however, will have a shift rows and mix columns operation applied to it.

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.mix_columns_output
attack = cwa.cpa(project, leak_model)
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

## Attacking from the other end

As we've seen, there's not much point to advancing past the SBox for our software AES attack. What about attacking from the other end? While the other leakage models didn't really any advantages over the SBox output for a software AES attack, there's actually a big reason why we'd like to try an attack from the other end of AES - we'd be able to use the ciphertext instead of the plaintext. While we know both the plaintext and ciphertext for our lab examples, this might not always be the case. For example, what if the target was encrypting messages and sending them to another device? We could monitor the communication lines to learn the ciphertext, but learning the plaintext isn't quite as simple.

Looking at a block diagram of AES, we can see that the last block is actually different from the rest of the blocks:

![](images/AES_Encryption.png)

The first thing to notice is that there's no MixColumns operation for the last round, which makes it a little easier to attack than across the first round. We can basically just repeat our SBox attack and recover the key, as can be seen with the LastroundHW leakage model:

```python
class LastroundHW(AESLeakageHelper):
    name = 'HW: AES Last-Round State'
    def leakage(self, pt, ct, key, bnum):
        # HD Leakage of AES State between 9th and 10th Round
        # Used to break SASEBO-GII / SAKURA-G
        st10 = ct[self.INVSHIFT_undo[bnum]]
        st9 = inv_sbox(ct[bnum] ^ key[bnum])
        return st9

    def process_known_key(self, inpkey):
        return key_schedule_rounds(inpkey, 0, 10)
```

Let's see if we can get this model to work. Capture some traces near the end of the AES operation (remember we're attacking at the last round),

and we can repeat the attack with the new leakage model:

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.last_round_state
attack = cwa.cpa(project, leak_model)
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

You should see that the attack is really confident about a key, but it doesn't seem to be the right one. The block diagram shown above doesn't tell the whole story. The AddRoundKey operation doesn't keep adding the same key. Instead, a transformation is applied to the key for each round and that is added to the state instead. The leakage model includes a method called `key_schedule_rounds()` that we can use to transform our last round key into the first round key that we would recover normally:

In [ ]:
input_key = leak_model.key_schedule_rounds(results.key_guess(), 10, 0)